In [4]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain.llms import OpenAI




In [5]:
GENERATIONS_PATH = "../../datasets/error_analysis/baseline_zero_shot_gpt3_50_50_merged.csv"
MODEL = "gpt-4-0613"


In [6]:
data = pd.read_csv(GENERATIONS_PATH)

data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output,structured_explanation,structured_explanation_v2
0,0,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.793Z,2023-10-22T13:05:59.340Z,Are the following two products the same?\nProd...,False.,False,True,FN,30%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
1,1,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.216Z,2023-10-22T13:05:58.791Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.825Z,2023-10-22T13:05:58.212Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
3,3,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.322Z,2023-10-22T13:05:57.823Z,Are the following two products the same?\nProd...,True,True,True,NONE,95%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:56.925Z,2023-10-22T13:05:57.320Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.2"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.1"",""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.654Z,2023-10-22T13:03:56.530Z,Are the following two products the same?\nProd...,False,False,False,NONE,0%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
217,217,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.074Z,2023-10-22T13:03:55.653Z,Are the following two products the same?\nProd...,False.,False,False,NONE,50%,70%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
218,218,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.615Z,2023-10-22T13:03:55.071Z,Are the following two products the same?\nProd...,False.,False,False,NONE,10%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
219,219,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.077Z,2023-10-22T13:03:54.614Z,Are the following two products the same?\nProd...,False.,False,False,NONE,25%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."


In [7]:
task_structured_explanation = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95" "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00" "values":["missing","Graphite"]}]"""

task_structured_explanation_v2 = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95", "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00", "values":["missing","Graphite"]}]"""


task_structured_explanation_v3 = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
attribute:brand, importance:0.05,values:[Logitech,Logitech]
attribute:model,importance:0.95,values:[G500,MX Master 3S]
attribute:color,importance:0.00,values:[missing,Graphite]"""




STEP_EXPLANATION_STRUCTURED_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", "{task_structured_explanation}"),
        ])




In [8]:
# if "structured_explanation" not in data.columns:
    # data["structured_explanation"] = ""
# if "structured_explanation_v2" not in data.columns:
    # data["structured_explanation_v2"] = ""
if "structured_explanation_v2_gpt4" not in data.columns:
    data["structured_explanation_v2_gpt4"] = ""

In [9]:
def get_structured_explanation(start, end):
    if (MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL, temperature=0)
    else:
        llm = ChatOpenAI(model_name=MODEL, temperature=0)
    chain = LLMChain(llm=llm, prompt=STEP_EXPLANATION_STRUCTURED_PROMPT)
    for i in range(start, end):
        print(i)
        user_prompt = data["prompt"][i]
        ai_answer = data["completion"][i]
        try:
            output = chain.run(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2)
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_EXPLANATION_STRUCTURED_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        data["structured_explanation_v2_gpt4"][i] = output



In [10]:
get_structured_explanation(0, len(data))

0
System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2: Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none

Answer with 'True' or 'False'.
AI: False.
Human: Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more sp

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_1316/3986336907.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2_gpt4"][i] = output


In [11]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output,structured_explanation,structured_explanation_v2,structured_explanation_v2_gpt4
0,0,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.793Z,2023-10-22T13:05:59.340Z,Are the following two products the same?\nProd...,False.,False,True,FN,30%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","[{""attribute"":""title"",""importance"":""0.5"",""valu..."
1,1,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.216Z,2023-10-22T13:05:58.791Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",...",
2,2,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.825Z,2023-10-22T13:05:58.212Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",...",
3,3,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.322Z,2023-10-22T13:05:57.823Z,Are the following two products the same?\nProd...,True,True,True,NONE,95%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...",
4,4,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:56.925Z,2023-10-22T13:05:57.320Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.2"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.1"",""...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.654Z,2023-10-22T13:03:56.530Z,Are the following two products the same?\nProd...,False,False,False,NONE,0%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...",
217,217,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:55.074Z,2023-10-22T13:03:55.653Z,Are the following two products the same?\nProd...,False.,False,False,NONE,50%,70%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",...",
218,218,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.615Z,2023-10-22T13:03:55.071Z,Are the following two products the same?\nProd...,False.,False,False,NONE,10%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...",
219,219,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:03:54.077Z,2023-10-22T13:03:54.614Z,Are the following two products the same?\nProd...,False.,False,False,NONE,25%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",...",


In [12]:
data.to_csv(GENERATIONS_PATH, index=False)
